In [ ]:
import diagonal_b6 as b6

# !conda install pandas -y
# !conda install matplotlib -y

import pandas as pd

In [ ]:
# connect to the world, in this instance Nottingham
grpc_address = "localhost:8002"
w = b6.connect_insecure(grpc_address)

In [ ]:
LINCOLN_CIRCUS_WAY_ID = 118144013
LINCOLN_CIRCUS_BENCH_NODE_ID = 2512226339

## the world?
One of the huge benefits of bedrock is that it allows you to build up a query or analysis without evaluating it. 
You evaluate your query, for example, via applying `w` to it. 
You will see this across the examples as `w(query)`

# Searching

b6 lets you search the world of osm in whatever data you have loaded. 
You do this via OSM tags (https://wiki.openstreetmap.org/wiki/Tags). Not all tags are "searchable", the ones that are have a `#` up front. Here are ones we have made searchable by default:

	"amenity":   "#amenity"
	"barrier":   "#barrier"
	"boundary":  "#boundary"
	"bridge":    "#bridge"
	"building":  "#building"
	"highway":   "#highway"
	"landuse":   "#landuse"
	"leisure":   "#leisure"
	"natural":   "#natural"
	"network":   "#network"
	"place":     "#place"
	"railway":   "#railway"
	"route":     "#route"
	"shop":      "#shop"
	"water":     "#water"
	"waterway":  "#waterway"
	"fhrs:id":   "@fhrs:id"
	"wikidata":  "@wikidata"
	"wikipedia": "@wikipedia"

In [ ]:
# find all the parks - https://wiki.openstreetmap.org/wiki/Tag:leisure%3Dpark
parks = b6.find( b6.tagged("#leisure","park") )

# access the metadata via `all_tags()`
for osmid,park in w(parks):
    print( f"{osmid}\n{park.all_tags()}" )

In [ ]:
# or if you knew the OSM ID of a park you could find it that way
lincoln_circus = b6.find_area( b6.osm_way_area_id(LINCOLN_CIRCUS_WAY_ID) )

print( w(lincoln_circus).all_tags() )

In [ ]:
# you can also search for a bench in the park
bench = b6.find_point( b6.osm_node_id(LINCOLN_CIRCUS_BENCH_NODE_ID ))

print( w(bench).all_tags() )

In [ ]:
# or if you didn't know what other features were in the park, 
# you could limit your search to what is in the park area
contained_in_lincoln_circus = b6.find( b6.intersecting( b6.find_feature(lincoln_circus) ) )

for osmid, feature in w(contained_in_lincoln_circus):
    print( f"{osmid}\n{feature.all_tags()}" )

Note that searching for what is in the park yields different types of features. 

- It yields `nodes`, which in this instance are benches. 
- It yields `ways`, which `bedrock` splits into `paths` and `areas`
    - `areas` are ways which represent areas, and the rest are `paths`       
- It also yields `access-path` which is also a `path` - these are paths we have automatically generated to ensure all features are connected to the network. This is helpful, for example if you want to compute routes.

In [ ]:
# you could be more precise and only search for `points` within the park
points_in_lincoln_circus = b6.find_points( b6.intersecting( b6.find_feature(lincoln_circus) ) )

for osmid, point in w(points_in_lincoln_circus):
    print( f"{osmid}\n{point.all_tags()}" )

In [ ]:
# you could be more precise and only search for `areas` within the park
areas_in_lincoln_circus = b6.find_areas( b6.intersecting( b6.find_feature(lincoln_circus) ) )

for osmid, area in w(areas_in_lincoln_circus):
    print( f"{osmid}\n{area.all_tags()}" )

In [ ]:
# you could be more precise and only search for `paths` within the park
paths_in_lincoln_circus = b6.find_paths( b6.intersecting( b6.find_feature(lincoln_circus) ) )

for osmid, path in w(paths_in_lincoln_circus):radius_m
    print( f"{osmid}\n{path.all_tags()}" )

###  Filter

You can use `.filter` to narrow down your search to something more specific.
You can also then apply `.map` to grab the value of a tag.
When grabbing the value of a tag you have four options:
- `get` which gets you whatever the value is
- `get_string`
- `get_int`
- `get)

In [ ]:
# you can search for features and filter to only get features that contain a certain tag
contained_in_lincoln_circus = b6.find( b6.intersecting( b6.find_feature(lincoln_circus) ) )
amenities_in_lincoln_circus = contained_in_lincoln_circus.filter(lambda a: b6.has_key(a, "#amenity"))

for osmid, amenity in w(amenities_in_lincoln_circus):
    print( f"{osmid}\n{amenity.all_tags()}" )

In [ ]:
# more generally you can do a wide search, using the searchable tags,
# then do a filter for features that have other non-searchable tags, present. 
offices_with_levels = b6.find_areas( b6.tagged("#building","office") ) \
                          .filter(lambda b: b6.has_key(b, "building:levels"))

for osmid, office in w(offices_with_levels):
    print( f"{osmid}\n{office.all_tags()}" )

In [ ]:
# even more fun, you can extract the value of that tag
levels_in_offices = offices_with_levels.map(lambda b: b6.get_int(b, "building:levels"))

for osmid, levels in w(levels_in_offices):
    print( f"{osmid} -->{levels} levels" )

In [ ]:
# you can use that to then analyse/categorise/filter by value as you need, using python
df = pd.DataFrame(w(levels_in_offices),
                  columns =["office","number_of_levels"]
                 ).set_index("office")

df.plot.hist()

# Compute

In [ ]:
lincoln_circus = b6.find_area( b6.osm_way_area_id(LINCOLN_CIRCUS_WAY_ID) )

In [ ]:
# you can find the closest pub to the park, within 1000 metres
pub = lincoln_circus.closest("walk", 1000.0, b6.tagged("#amenity", "pub"))
print( w(pub).all_tags() )

In [ ]:
# you can find the distance to the closest pub within 1000 metres
distance = lincoln_circus.closest_distance("walk", 1000.0, b6.tagged("#amenity", "pub"))
print( f"{w(distance)} metres" )

In [ ]:
# or you can find all the pubs within 1000 metres
pubs = lincoln_circus.reachable("walk", 1000.0, b6.tagged("#amenity","pub"))
for osmid, pub in w(pubs):
    print( f"{osmid}\n{pub.all_tags()}" )

In [ ]:
# if you wanted to map these elsewhere, you could export as geojson
print( w( b6.to_geojson_collection(pubs) ) )

In [ ]:
# if you just want to know how many pubs are within a certain distance, you can just ask for that
number_of_pubs = lincoln_circus.reachable("walk", 1000.0, b6.tagged("#amenity","pub")).count()
print( f"{w(number_of_pubs)} pubs within a 1000 metre walk" )

In [ ]:
# if you wanted to know the paths to reach the different pubs you can compute that too
# this will find each `path` in the data that was used to arrive at any of the reachable pubs,
# and give you a count of how many times that path was used.
paths = lincoln_circus.paths_to_reach("walk", 1000.0, b6.tagged("#amenity","pub"))

for osmid, use_count in w(paths):
    print( f"{osmid} -> used {use_count} times" )

## Change the world and compute

The `b6.with_change` function lets you run an analysis having changed attributes in the built environment. 

### Adding tags and evaluating the change

In [ ]:
# Compute how many parks
# leisure:park = https://wiki.openstreetmap.org/wiki/Tag:leisure%3Dpark
# are within 1km of a school, for all schools

schools = b6.find_areas( b6.tagged("#amenity","school") )
reachable_parks_from_schools = schools.map( lambda school: school.reachable("walk",
                                                                            1000.0,
                                                                            b6.tagged("#leisure","park")) \
                                                                   .count() 
                                          )

In [ ]:
df = pd.DataFrame(w(reachable_parks_from_schools),
                  columns =["school","number_of_parks"]
                 ).set_index("school")
df.plot.hist(bins=max(df["number_of_parks"])+1)

In [ ]:
# now, imagine you want to know what happens if all parking lots, 
# amenity:parking - https://wiki.openstreetmap.org/wiki/Tag:amenity%3Dparking denoted as areas,
# are turned into parks

parkinglots = b6.find_areas(b6.tagged("#amenity","parking"))
parkinglots_to_parks = b6.add_tags( parkinglots.map( lambda parkinglot: b6.tag("#leisure","park") )
                                    )

change = pd.DataFrame(w(b6.with_change(parkinglots_to_parks, lambda: reachable_parks_from_schools)),
                      columns =["school","number_of_parks"]
                     ).set_index("school")


In [ ]:
data = df.merge(change, right_index=True, left_index=True, suffixes=["_before","_after"])
data.plot.hist(alpha=0.5, bins=max(data["number_of_parks_after"])+1)